## Train h1 classifier

We define sub category based on classifier_similarity. And train each subcategory only inside that sub category.
We call the lowest category h1, higher category as h2.

In [ ]:
import os
import sys
import json

import sys


import akagi
print("akagi version:", akagi.__version__)
from akagi.data_sources import RedshiftDataSource, S3DataSource
from akagi.iterator import FileFormat

import numpy as np

import pandas as pd
import glob

In [ ]:
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

In [ ]:
# created by classifier_similarity.ipynb
VALID_DISTANCE_DICT_PATH=""

In [ ]:
#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()


### Downlaod similarity df from S3

In [ ]:
!aws s3 cp 

## Multiple Model Train

In [ ]:
from one_vs_all import create_multiple_model_trainer

def create_multimodeltrainer():
    return create_multiple_model_trainer(base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, traingen=TRAIN_DATAGEN, validgen=VALID_DATAGEN, extractor_path=CODE_EXTRACTOR)

In [ ]:
from modelutils import dir2filedict

In [ ]:
multitrainer = create_multimodeltrainer()

In [ ]:
multitrainer.setup_filedict(dir2filedict(TRAIN_DATA_DIR), dir2filedict(VALID_DATA_DIR))
catkeys = sorted(list(multitrainer.train_files_dict.keys()))

In [ ]:
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
DIST_THRESHOLD = 1.5

In [ ]:
def train_subs(keys):
    for targetkey in keys:
        similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)
        multitrainer.train_sub(targetkey, similarkeyset,  eachepochs=10)

## Training

In [ ]:
train_subs(catkeys[0:5])

In [ ]:
train_subs(catkeys[5:65])

In [ ]:
152+17

In [ ]:
train_subs(catkeys[152:169])

## Obsolete

In [ ]:
#targetkey = '93'
targetkey = '94'


In [ ]:
similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)

In [ ]:
# temp workaround: remove vc
similarkeyset = similarkeyset - set(['137_170', '142_66_95_31_50_145', '146_149_53', '16_32_87'])

In [ ]:
multitrainer.train_sub(targetkey, similarkeyset,  eachepochs=10)

### Create code model and put to S3

In [ ]:
targetcats = catkeys[0:65] + [catkeys[152]]

In [ ]:
DIST_DIR=''

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
import tqdm

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_" + key) for key in tqdm.tqdm(targetcats)]

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_full_path

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight, _, _ = best_code_full_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to("h1_"+key) for key in targetcats]

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
!aws s3 sync --exclude "*" --include "code*" {DIST_DIR} {S3PATH}  

## Manual Sub Category Training

Sub category training by hand.

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename

In [ ]:
# keyset = set(['108_72', '109', '93', '17', '18'])
# keyset = set(['105', '108_72', '109', '125', '154', '156_79', '162', '17', '175','177', '18', '21', '39', '4', '64', '69', '8', '82', '93'])

# for 108_72
keyset = set(['10', '105', '108_72', '109', '111', '137_170', '147', '151', '156_79','162', '17', '18', '20', '21', '39', '4', '40', '49', '5', '57', '8','83', '92', '93'])

In [ ]:
multitrainer = create_multimodeltrainer()
multitrainer.setup_filedict(TRAIN_DATA_DIR, VALID_DATA_DIR)
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=3, retrainings=2)

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_108_72")

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=10, retrainings=5)

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_93")

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=1, retrainings=1, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=4, retrainings=3, removecheckpoint=False)

In [ ]:
multitrainer.remove_checkpoint('h1_93')

In [ ]:
multitrainer.train_sub('17', keyset,  eachepochs=3, retrainings=2)

In [ ]:
multitrainer.train_sub('17', keyset,  eachepochs=4, retrainings=1)

In [ ]:
multitrainer.train_sub('17', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('18', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=3, retrainings=1)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=3, retrainings=3)

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=10, retrainings=3)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('18', keyset,  eachepochs=10, retrainings=1)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=3, retrainings=3)

## Create code model and send to S3

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename

In [ ]:
for key in keyset:
    best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_" + key)

In [ ]:
DIST_DIR=''

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_full_path

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight, _, _ = best_code_full_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to("h1_" + key) for key in keyset]

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
# !aws s3 sync --dryrun --exclude "*" --include "code*" {DIST_DIR} {S3PATH}    
!aws s3 sync --exclude "*" --include "code*" {DIST_DIR} {S3PATH}    

# Obsolete logs

In [ ]:
multitrainer.train_sub('18', keyset,  eachepochs=1, retrainings=1)

In [ ]:
multitrainer.train_sub('18', keyset,  eachepochs=4, retrainings=1)

In [ ]:
multitrainer.remove_checkpoint('h1_17')

In [ ]:
multitrainer.remove_checkpoint('h1_18')

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_17")

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_18")

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=4, retrainings=3, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('109', keyset,  eachepochs=10, retrainings=1, removecheckpoint=False)

In [ ]:
catkeys = multitrainer.vc.keys()

In [ ]:
multitrainer.remove_checkpoint('h1_109')

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=1, retrainings=1, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=8, retrainings=2, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('108_72', keyset,  eachepochs=4, retrainings=2, removecheckpoint=False)

In [ ]:
multitrainer.remove_checkpoint('h1_108_72')

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_93")

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_109")

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_108_72")

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_109")

### Add ramen trial (39)

pasta (93) and ramen is sometime mixed. Can I separate ramen when add this as h2 category?

In [ ]:
keyset = set(['108_72', '109', '93', '17', '18', '39'])

In [ ]:
multitrainer = create_multimodeltrainer()

In [ ]:
multitrainer.setup_filedict(TRAIN_DATA_DIR, VALID_DATA_DIR)

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=1, retrainings=1, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=1, retrainings=1, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('93', keyset,  eachepochs=4, retrainings=2, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('39', keyset,  eachepochs=1, retrainings=1, removecheckpoint=False)

In [ ]:
multitrainer.train_sub('39', keyset,  eachepochs=4, retrainings=2, removecheckpoint=False)

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_93")

In [ ]:
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, "h1_39")